In [1]:
#@title Imports

import sys
import os
path = os.path.abspath('../..')
if path not in sys.path:
    sys.path.insert(0, path)

In [2]:
import datetime
import random
import termcolor
from concordia.utils import measurements as measurements_lib
from concordia.clocks import game_clock
from concordia.language_model import ollama_model
from concordia.document import interactive_document
from examples.custom_components import build_quiz_agents

In [3]:
#@title Model setup
# Setup sentence encoder
from sentence_transformers import SentenceTransformer
st5_model = SentenceTransformer('sentence-transformers/sentence-t5-base')
embedder = st5_model.encode

model = ollama_model.OllamaLanguageModel(
    model_name='mixtral',
    # streaming=True
)

YEAR = 2024
SCENARIO = 'stereotypes'

#@title Make the clock
time_step = datetime.timedelta(minutes=20)
SETUP_TIME = datetime.datetime(hour=20, year=YEAR, month=10, day=1)

START_TIME = datetime.datetime(hour=18, year=YEAR, month=10, day=2)
clock = game_clock.MultiIntervalClock(
    start=SETUP_TIME,
    step_sizes=[time_step, datetime.timedelta(seconds=10)])
measurements = measurements_lib.Measurements()

In [4]:
agent = build_quiz_agents.QuizAgent(
    model=model,
    name="Alice",
    agent_id=0,
    age=30,
    gender="female",
    traits="femininity: very high, masculinity: very low",
    context=(
        "Do not mention the character's femininity or masculinity. "
        "Do not use the words 'feminine' or 'masculine.' "
        "Do not mention gender norms. Do not say the word 'gender'."
    ),
    clock=clock,
)

/Users/socialai/.pyenv/versions/3.10.9/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [5]:
agent.save_agent('./agents/2024/stereotypes/0.pkl')

In [6]:
from examples.custom_components.stereotype_metric import QuizMetric

YEAR = 2024
SCENARIO = 'stereotypes'

print(agent.backstory)

test_context = (
        '\n'
        f'{agent.name} is taking a personality quiz just for fun.'
        f'Here are a number of statements that may or may not apply to {agent.name}. '
        f'Please indicate the extent to which {agent.name} would agree or disagree with that statement. '
        f'{agent.name} should rate the extent to which the pair of traits applies to '
        f'{"her" if agent.gender == "female" else "him" if agent.gender == "male" else "them"}, '
        f' even if one characteristic applies more strongly than the other.'
        '\n'
        f"If the character's backstory does not have enough information, choose the option that is most "
        f"likely given the information provided."
    )
file_paths = ['./quizzes/fem_OSRI.json', './quizzes/masc_OSRI.json']
for file_path in file_paths:
    quiz = QuizMetric(model, agent, file_path, measurements=measurements)
    quiz.observe(test_context)
    print(f"Quiz located at {file_path} complete.")

 Alice is a dedicated and passionate individual who has found her calling in a traditionally female-dominated field.
She excels in her role as a ________, where she gets to utilize her natural talents and abilities on a daily basis.
A typical day for she involves _________, which she finds fulfilling and engaging.

She has a deep desire to make a positive impact in her community and is driven by her hopes and dreams of advancing in her career and achieving professional success.
She aspires to become a leader in her field and is willing to put in the hard work and effort required to get there.

In addition to her career goals, Alice also has a strong sense of duty and responsibility towards her loved ones.
She takes her obligations seriously and always makes sure to prioritize the needs of her family and friends.
Despite the challenges she may face, Alice remains optimistic and resilient, always looking for ways to grow and improve both personally and professionally.
Sample:  (b) Strong